In [74]:
import json
import http.client
import pandas as pd
# from dotenv import load_dotenv
import os
# load_dotenv()  # take environment variables from .env.

In [75]:
def get_token(access_key): 
    conn = http.client.HTTPSConnection("www.ura.gov.sg")
    payload = ''
    headers = {
    'AccessKey': access_key
    }
    conn.request("GET", "/uraDataService/insertNewToken.action", payload, headers)
    res = conn.getresponse()
    data = res.read()
    return json.loads(data.decode("utf-8"))['Result']

In [76]:
def get_result(token, access_key, route):
    conn = http.client.HTTPSConnection("www.ura.gov.sg")
    payload = ""
    headers = {
        "AccessKey": access_key,
        "Token": token,
        "User-Agent": "PostmanRuntime/7.26.8",
    }
    conn.request("GET", route, payload, headers)
    res = conn.getresponse()
    data = res.read()
    data = json.loads(data.decode("utf-8"))['Result']
    # data = pd.json_normalize(data = data, record_path='transaction', meta = ['street', 'project'])
    return data


def get_all_results(token, access_key, routes):
    result = []
    for route in routes:
        result.extend(get_result(token, access_key, route))
    return result


In [77]:

def get_all_ura():
    private_transactions_routes = [
        "/uraDataService/invokeUraDS?service=PMI_Resi_Transaction&batch=" + str(i)
        for i in range(1, 5)
    ]
    private_rental_routes = [
        f"/uraDataService/invokeUraDS?service=PMI_Resi_Rental&refPeriod={q}"
        # start from 14q1 to 23q1
        for q in [f"{y}q{i}" for y in range(14, 24) for i in range(1, 5)]
    ]  

    planning_decisions_routes = [
        "/uraDataService/invokeUraDS?service=Planning_Decision&year=" + str(i)
        for i in range(2000, 2024)
    ]

    access_key = "480de617-6aee-4c71-a04a-3b6c3a9596b2"
    token = get_token(access_key)

    # get_all_results(token, access_key, private_transactions_routes).to_csv(
    #     "../data/private_transactions.csv", index=False
    # )
    # get_all_results(token, access_key, private_rental_routes).to_csv(
    #     "../data/private_rental.csv", index=False
    # )
    # get_all_results(token, access_key, planning_decisions_routes).to_csv(
    #     "../data/planning_decisions.csv", index=False
    # )

    df_private_transactions = pd.json_normalize(
        data = get_all_results(token, access_key, private_transactions_routes), 
        record_path='transaction', 
        meta = ['street', 'project', 'marketSegment']
        )

    df_private_rental = pd.json_normalize(
        data = get_all_results(token, access_key, private_rental_routes), 
        record_path='rental', 
        meta = ['street', 'project']
        )

    df_planning_decisions = get_all_results(token, access_key, planning_decisions_routes)

    return df_private_transactions, df_private_rental, df_planning_decisions


In [78]:
df_private_transactions, df_private_rental, df_planning_decisions = get_all_ura()

In [79]:
df_private_transactions

,area,floorRange,noOfUnits,contractDate,typeOfSale,price,propertyType,district,typeOfArea,tenure,nettPrice,street,project,marketSegment
0,524.3,-,1,0220,3,5500000,Semi-detached,05,Land,Freehold,NaN,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR
1,308,-,1,0918,3,5000000,Semi-detached,05,Land,Freehold,NaN,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR
2,314,-,1,0618,3,4750000,Semi-detached,05,Land,Freehold,NaN,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR
3,308,-,1,0921,3,5200000,Semi-detached,05,Land,Freehold,NaN,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR
4,159.3,-,1,0320,3,2630000,Terrace,05,Land,Freehold,NaN,NEO PEE TECK LANE,LANDED HOUSING DEVELOPMENT,RCR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131208,215,01-05,1,0819,3,1100000,Apartment,27,Strata,Freehold,NaN,GOODLINK PARK,GOODLINK PARK,OCR
131209,295.3,-,1,0921,3,3400000,Terrace,27,Land,Freehold,NaN,GOODLINK PARK,GOODLINK PARK,OCR
131210,193,01-05,1,0521,3,1300000,Apartment,27,Strata,Freehold,NaN,GOODLINK PARK,GOODLINK PARK,OCR
131211,175,-,1,0322,3,3000000,Terrace,27,Land,Freehold,NaN,GOODLINK PARK,GOODLINK PARK,OCR


In [73]:
df_private_rental


,areaSqm,leaseDate,propertyType,district,areaSqft,noOfBedRoom,rent,street,project
0,190-200,0318,Non-landed Properties,15,2100-2200,3,4700,ELLIOT ROAD,ELLIOT AT THE EAST COAST
1,250-260,0318,Non-landed Properties,15,2700-2800,4,6800,ELLIOT ROAD,ELLIOT AT THE EAST COAST
2,240-250,0318,Non-landed Properties,15,2600-2700,4,6000,ELLIOT ROAD,ELLIOT AT THE EAST COAST
3,190-200,0318,Non-landed Properties,15,2100-2200,3,5000,ELLIOT ROAD,ELLIOT AT THE EAST COAST
4,100-110,0318,Non-landed Properties,23,1100-1200,2,2300,HILLVIEW AVENUE,MERAWOODS
...,...,...,...,...,...,...,...,...,...
477675,60-70,0123,Non-landed Properties,08,600-700,3,5300,PERUMAL ROAD,UPTOWN @ FARRER
477676,40-50,0123,Non-landed Properties,08,500-600,NA,4000,PERUMAL ROAD,UPTOWN @ FARRER
477677,40-50,0123,Non-landed Properties,08,500-600,2,4000,PERUMAL ROAD,UPTOWN @ FARRER
477678,40-50,0223,Non-landed Properties,08,500-600,2,4000,PERUMAL ROAD,UPTOWN @ FARRER


In [ ]:
df = pd.json_normalize(data = data, record_path='transaction', meta = ['street', 'project', 'marketSegment'])

In [25]:
df_planning_decisions

,decision_date,address,submission_desc,decision_type,appl_type,mkts_lotno,dr_id,submission_no,decision_no
0,01/01/2000,ORCHARD ROAD,PROPOSED OUTDOOR REFRESHMENT AREA FOR UNIT #01-02,Written Permission,Addition/Alteration to Buildings (Others),"TS21 551, TS21 554PT, TS21 787, TS21 788, TS21...",55888b,071098-27Z3-Z000,P071098-27Z3-Z000
1,03/01/2000,CORONATION ROAD WEST,PROPOSED AMENDMENT TO APPROVED ERECTION OF 6 U...,Written Permission,Amendment to New Erection,MK04 110,55911,120496-34D2-A022,P120496-34D2-A022
2,03/01/2000,BEECHWOOD GROVE,PROPOSED AMENDMENT TO APPROVED ERECTION OF 2 U...,Written Permission,Amendment to New Erection,"MK13 2903, MK13 2904",53075,300196-24H3-A011,P300196-24H3-A011
3,03/01/2000,VALLEY ROAD,SUBDIVISION OF LAND INTO 2 PLOTS,Written Permission,Subdivision of Land,MK22 5115,46971b,071299-07I1-Z000,P071299-07I1-Z000
4,03/01/2000,554 SEMBAWANG ROAD,PROPOSED ADDITIONS AND ALTERATIONS TO THE EXIS...,Written Permission,Addition/Alteration to Landed Housing,MK19 673,47077b,021299-24D1-Z000,P021299-24D1-Z000
...,...,...,...,...,...,...,...,...,...
175845,17/03/2023,"42,41,40,39,38,37,36,35,34,33,32,31,30,29 MENG...",PROPOSED SUBDIVISION OF LAND ON LOT 551T MK13 ...,Written Permission,Subdivision of Land,MK13 00551T,284159,170223-12D1-Z000,P170223-12D1-Z000
175846,17/03/2023,1 ANDREW ROAD,PROPOSED NEW ERECTION OF A 2-STOREY ENVELOPE C...,Written Permission,New Erection,MK17 07664W,284164,050322-55G2-Z000,P050322-55G2-Z000
175847,17/03/2023,143 CECIL STREET,PROPOSED CHANGE OF USE OF UNIT #02-01 FROM SHO...,Written Permission,Change of Use,TS02 00257C,284182,070123-01D1-Z000,P070123-01D1-Z000
175848,17/03/2023,3 ANSON ROAD,PROPOSED CHANGE OF USE OF OFFICE UNIT #07-02 T...,Written Permission,Change of Use,"TS30 00145M,TS23 00234M,TS23 01175X",284176,151222-23J2-Z000,P151222-23J2-Z000
